# Optical flow

In [ ]:
import cv2
import numpy as np

In [2]:
# detect 10 corners and track those
corner_track_params = dict(maxCorners = 10,qualityLevel=0.3,minDistance=7,blockSize=7) 
# maxCorners is that how many corners we want to detect the quality one
# qualityLevel, minDistance, minDistance this are default values wecan tune them

In [3]:
# lk_params is lucas kanade optical flow function
lk_params = dict(winSize=(200,200), maxLevel = 2, criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03))
# winsize=(200,200)is trad of if choose small window size it too much noise to track them and we can't detect larger motion but if we choose larger winsize we can't detect small motion

# maxLevel is lucas kanade algorithm use the image pyramid method it allow us to find optical flow at various resolution of image 
# ex. Level 0 is orignal image 
# Level 1 is 1/2 resolution image
# Level 2 is 1/4 resolution image
# Level 3 is 1/8 resolution image
# Level 4 is 1/16 resolution image

# criteria IS we are providing two criteria to perform lucas kanade optical flow, we providing maximum no. of iteration
# and 10 is for cv2.TERM_CRITIERIA_COUNT. and 0.03 is for cv2.TERM_CRITERIA_EPS. EPS means episolon
# more iteration means more exhaustive search for the points
# so that's to do if the counts on how many iteration are you going to be looking for these points in current frame vs the prevoius frame
# what these does it exchanging speed of your tracking vs the accuracy of your tracking
# 10,0.03 is we have to tune for our video

In [4]:
cap = cv2.VideoCapture(0)

ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# POINTS TO TRACK
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None, **corner_track_params) # we have to put **before the corner_track_params because it is how put dictionary ito the function

# actual drawing line in video then tracking points
mask = np.zeros_like(prev_frame)

while True:
    
    ret, frame = cap.read() # frame is the currnet frame
    
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,frame_gray,prevPts,None,**lk_params) # cv2.calcOpticalFlowPyrLK in PyrLK is lucas kanade pyramid
    
    # status will return status array
    # it output what known as a status vector where each element of the vector is set to one if the flow for the corresponding features has been found otherwise it set to zero
    
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1] # matching them up based on the index location
    # status vector is set to 1 if the flow for the corresponding features has been found
    # essentially this is connecting where the prevoius points were to the next set of points
    
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        # Lines will be drawn using the mask created from the first frame
        mask = cv2.line(mask, (x_new,y_new), (x_prev,y_prev),(0,255,0),3)
        
        # Draw red circles at corner points
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)
        
    img = cv2.add(frame,mask)
    cv2.imshow('tracking',img)
    
    k = cv2.waitKey(30) & 0xFF
    if k == 27:
        break
        
    # now we have to update the current frame to be the prevoius frame for next iteration
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2) # we have to reshpae good_new.reshape(-1,1,2) this because this way cv2.calcOpticalFlowPyrLK this function accept
    
cv2.destroyAllWindows()
cap.release()

# Dense Optical Flow in OpenCV

calcOpticalFlowFarneback(prev, next, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags) -> flow

This function computes a dense optical flow using the Gunnar Farneback's algorithm.

Here are the parameters for the function and what they represen

* prev first 8-bit single-channel input image.
* next second input image of the same size and the same type as prev.
* flow computed flow image that has the same size as prev and type CV_32FC2.
* pyr_scale parameter, specifying the image scale (\<1) to build pyramids for each image
    * pyr_scale=0.5 means a classical pyramid, where each next layer is twice smaller than the previous one.
    
* levels number of pyramid layers including the initial image; levels=1 means that no extra layers are created and only the original images are used.
* winsize averaging window size
    * larger values increase the algorithm robustness to image
* noise and give more chances for fast motion detection, but yield more blurred motion field.
* iterations number of iterations the algorithm does at each pyramid level.
* poly_n size of the pixel neighborhood used to find polynomial expansion in each pixel
    * larger values mean that the image will be approximated with smoother surfaces, yielding more robust algorithm and more blurred motion field, typically poly_n =5 or 7.
* poly_sigma standard deviation of the Gaussian that is used to smooth derivatives used as a basis for the polynomial expansion; for poly_n=5, you can set poly_sigma=1.1, for poly_n=7, a good value would be poly_sigma=1.5.

In [1]:
import cv2
import numpy as np

In [2]:
cap = cv2.VideoCapture(0)

ret, frame1 = cap.read()

prevsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255 # hsv is hue stauration value

while True:
    
    ret, frame2 = cap.read()
    
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    
    # calulate the optical flow
    # REFER BOOK IN THAT ALL THE DETAILS ARE WRITTEN
    flow = cv2.calcOpticalFlowFarneback(prevsImg,nextImg,None,0.5,3,15,3,5,1.2,0) # default parameter
    
    mag, ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1],angleInDegrees=True) # cartigian cooridinate for vectoers
    # flow[:,:,0] for every pixel in x and y choose horizontal(0)
    # flow[:,:,1] for every pixel in x and y choose vertical(0)
    hsv_mask[:,:,0] = ang/2
    # hsv_mask[:,:,0] for all pixels we grab the hue
    # ang/2 this way we don't get so many hues and we actual looks half the hues it help distinguishinfo on left right up down a little clear so essemtially 360 degree from 180
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    # hsv_mask[:,:,2]it grab the actual values
    # to find the value based on the magnitude
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame',bgr)
    
    k = cv2.waitKey(10) & 0xFF
    
    if k == 27:
        break
        
    prevsImg =nextImg
    
cap.release()
cv2.destroyAllWindows()
        
